# Step-by-step YOLOv8 Workflow in Colab


Upload directory in google drive and connect drive


Below will be you directory structure :


/content/drive/MyDrive/HackByte_Dataset/
│
├── dataset/                # Contains train/val/test folders in YOLO format
│   ├── train/images, labels/
│   ├── val/images, labels/
│   └── test/images, labels/
│
├── ENV_SETUP/              # Conda-related stuff (ignored in Colab unless you're using Jupyter+Conda locally)
│
├── classes.txt
├── predict.py
├── train.py
├── visualise.py
└── yolo_params.yaml


 ## 1. Mount Google Drive

In [2]:
import os

In [3]:
# Replace the path below if your dataset folder is in another location inside Drive.
%cd /kaggle/input/data

/kaggle/input/data


In [4]:
# Verify Folder Contents
!ls

test  train  val


## 2. Install YOLOv8 (Ultralytics)

In [5]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 16.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

## checking gpu

In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

CUDA available: True
Device: cuda


In [7]:
from ultralytics.utils.torch_utils import select_device
device = select_device("cuda")
print(device)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
cuda:0


In [8]:
yaml_content = """
train: /kaggle/input/data/train
val: /kaggle/input/data/val
test: /kaggle/input/data/test
nc: 3
names: ['FireExtinguisher', 'ToolBox', 'OxygenTank']
"""

with open("/kaggle/working/data.yaml", "w") as f:
    f.write(yaml_content)


## 6. Train YOLOv8 Model

In [9]:
%cd /kaggle/working

/kaggle/working


You can run your own script:

In [10]:
code = '''
EPOCHS = 1
MOSAIC = 0.1
OPTIMIZER = "AdamW"
MOMENTUM = 0.2
LR0 = 0.001
LRF = 0.0001
SINGLE_CLS = False
BATCH_SIZE = 16  # Default batch size

import argparse
from ultralytics import YOLO
import os
from ultralytics.utils.torch_utils import select_device

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    # Model argument
    parser.add_argument("--model", type=str, required=True, help="Path to the YOLOv8 model file (e.g., yolov8n.pt, yolov8s.pt)")

    # epochs
    parser.add_argument("--epochs", type=int, default=EPOCHS, help="Number of epochs")

    # mosaic
    parser.add_argument("--mosaic", type=float, default=MOSAIC, help="Mosaic augmentation")

    # optimizer
    parser.add_argument("--optimizer", type=str, default=OPTIMIZER, help="Optimizer")

    # momentum
    parser.add_argument("--momentum", type=float, default=MOMENTUM, help="Momentum")

    # lr0
    parser.add_argument("--lr0", type=float, default=LR0, help="Initial learning rate")

    # lrf
    parser.add_argument("--lrf", type=float, default=LRF, help="Final learning rate")

    # single_cls
    parser.add_argument("--single_cls", type=bool, default=SINGLE_CLS, help="Single class training")

    # batch size
    parser.add_argument("--batch_size", type=int, default=BATCH_SIZE, help="Batch size for training")

    print("used device is : ")
    device = select_device("cuda")
    print(device)
    print("----------")

    args = parser.parse_args()
    this_dir = os.path.dirname(__file__)
    os.chdir(this_dir)

    # Load the specified YOLOv8 model
    model = YOLO(args.model)

    results = model.train(
        data=os.path.join(this_dir, "data.yaml"),
        epochs=args.epochs,
        device=0,
        single_cls=args.single_cls,
        optimizer=args.optimizer,
        lr0=args.lr0,
        lrf=args.lrf,
        momentum=args.momentum,
        batch=args.batch_size,  # Pass batch size
        verbose=True
    )



'''

with open('train.py', 'w') as f:
    f.write(code)

print("✅ train.py has been created successfully.")


✅ train.py has been created successfully.


In [11]:
%cd /kaggle/working

/kaggle/working


In [15]:
!python train.py --model yolov8m.pt --epochs 30 --batch_size 24 --optimizer AdamW --lr0 0.001 --lrf 0.001 --momentum 0.9 --mosaic 0.4


used device is : 
Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
cuda:0
----------
100%|███████████████████████████████████████| 49.7M/49.7M [00:00<00:00, 227MB/s]
Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/working/data.yaml, epochs=30, time=None, patience=100, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms

## 7. Run Prediction

In [16]:
code = '''
from ultralytics import YOLO
from pathlib import Path
import cv2
import os
import yaml

# Function to predict and save images
def predict_and_save(model, image_path, output_path, output_path_txt):
    results = model.predict(image_path, conf=0.5)
    result = results[0]
    img = result.plot()
    cv2.imwrite(str(output_path), img)
    with open(output_path_txt, 'w') as f:
        for box in result.boxes:
            cls_id = int(box.cls)
            x_center, y_center, width, height = box.xywh[0].tolist()
            f.write(f"{cls_id} {x_center} {y_center} {width} {height}\\n")

if __name__ == '__main__': 
    this_dir = Path(__file__).parent
    os.chdir(this_dir)
    with open(this_dir / 'data.yaml', 'r') as file:
        data = yaml.safe_load(file)
        if 'test' in data and data['test'] is not None:
            images_dir = Path(data['test']) / 'images'
        else:
            print("No test field found in data_params.yaml.")
            exit()
    
    if not images_dir.exists() or not images_dir.is_dir() or not any(images_dir.iterdir()):
        print(f"Images directory issue: {images_dir}")
        exit()

    detect_path = this_dir / "runs" / "detect"
    train_folders = [f for f in os.listdir(detect_path) if os.path.isdir(detect_path / f) and f.startswith("train")]
    if len(train_folders) == 0:
        raise ValueError("No training folders found")

    model_path = detect_path / train_folders[0] / "weights" / "best.pt"
    model = YOLO(model_path)

    output_dir = this_dir / "predictions"
    images_output_dir = output_dir / 'images'
    labels_output_dir = output_dir / 'labels'
    images_output_dir.mkdir(parents=True, exist_ok=True)
    labels_output_dir.mkdir(parents=True, exist_ok=True)

    for img_path in images_dir.glob('*'):
        if img_path.suffix.lower() not in ['.png', '.jpg', '.jpeg']:
            continue
        output_path_img = images_output_dir / img_path.name
        output_path_txt = labels_output_dir / img_path.with_suffix('.txt').name
        predict_and_save(model, img_path, output_path_img, output_path_txt)

    print(f"Predicted images saved in {images_output_dir}")
    print(f"Bounding box labels saved in {labels_output_dir}")
    print(f"Model parameters saved in {this_dir / 'data.yaml'}")
    metrics = model.val(data=this_dir / 'data.yaml', split="test")
'''

with open('predict.py', 'w') as f:
    f.write(code)

print("✅ predict.py has been created successfully.")


✅ predict.py has been created successfully.


In [17]:
!python predict.py


image 1/1 /kaggle/input/data/test/images/2000000003.png: 384x640 1 FireExtinguisher, 1 ToolBox, 60.6ms
Speed: 2.9ms preprocess, 60.6ms inference, 191.1ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /kaggle/input/data/test/images/2000000033.png: 384x640 1 OxygenTank, 25.1ms
Speed: 2.0ms preprocess, 25.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /kaggle/input/data/test/images/1000000076.png: 384x640 1 FireExtinguisher, 25.2ms
Speed: 2.1ms preprocess, 25.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /kaggle/input/data/test/images/000000115.png: 384x640 1 ToolBox, 25.2ms
Speed: 2.0ms preprocess, 25.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /kaggle/input/data/test/images/1000000061.png: 384x640 1 ToolBox, 25.2ms
Speed: 2.0ms preprocess, 25.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /kaggle/input/data/test/images/000000127.png: 384x6

## 8. Visualize Predictions